In [1]:
%env THEANO_FLAGS='device=cpu','floatX=float32'

import os
import os.path as osp

import numpy as np

import theano
import theano.tensor as T

from lasagne import *

%matplotlib nbagg
import matplotlib.pyplot as plt

from mldm import NNWatcher, Net

env: THEANO_FLAGS='device=cpu','floatX=float32'


/home/mborisyak/opt/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [17]:
class Discriminator(Net):
    def __init__(self, X_real, X_pseudo):

        self.input_l = layers.InputLayer(
            (None, 1, 28, 28),
            name='input layer'
        )
        
        conv1 = layers.Conv2DLayer(
            self.input_l,
            num_filters=16,
            filter_size=(3, 3),
            nonlinearity=nonlinearities.leaky_rectify,
            pad='same'
        )
        
        pool1 = layers.Pool2DLayer(
            conv1,
            pool_size=(2, 2)
        )
        
        conv2 = layers.Conv2DLayer(
            pool1,
            num_filters=16,
            filter_size=(3, 3),
            nonlinearity=nonlinearities.leaky_rectify
        )
        
        pool2 = layers.Pool2DLayer(
            conv2,
            pool_size=(2, 2)
        )
        
        conv3 = layers.Conv2DLayer(
            pool2,
            num_filters=32,
            filter_size=(3, 3),
            nonlinearity=nonlinearities.leaky_rectify
        )
        
        pool3 = layers.GlobalPoolLayer(
            conv3,
            pool_function = T.mean
        )
        
        out = layers.DenseLayer(
            pool3,
            num_units=1,
            nonlinearity=nonlinearities.sigmoid
        )
        
        self.net = layers.FlattenLayer(out)
        
        self.real_predictions = layers.get_output(
            self.net, inputs = {
                self.input_l : X_real
            }
        )
        
        self.pseudo_predictions = layers.get_output(
            self.net, inputs = {
                self.input_l : X_real
            }
        )

In [20]:
from theano.tensor.shared_randomstreams import RandomStreams

rng = RandomStreams()

In [27]:
class MNISTGenerator(Net):
    def __init__(self, batch_size = 32):
        noise_shape = (28, 28)
        X_noise = rng.normal(size=(batch_size, 1, ) + noise_shape, dtype='floatX')
        
        self.input_layer = layers.InputLayer(shape=(batch_size, 1, ) + noise_shape, input_var=X_noise)
        
        conv1 = layers.Conv2DLayer(
            self.input_layer,
            num_filters=16,
            filter_size=(3, 3),
            pad='same'
        )
        
        conv2 = layers.Conv2DLayer(
            conv1,
            num_filters=32,
            filter_size=(3, 3),
            pad='same'
        )
        
        conv3 = layers.Conv2DLayer(
            conv2,
            num_filters=32,
            filter_size=(3, 3),
            pad='same'
        )
        
        conv4 = layers.Conv2DLayer(
            conv3,
            num_filters=16,
            filter_size=(3, 3),
            pad='same'
        )
        
        conv5 = layers.Conv2DLayer(
            conv4,
            num_filters=1,
            filter_size=(3, 3),
            pad='same'
        )
        
        self.net = conv5
        
        self.X_pseudo = layers.get_output(self.net)

In [28]:
print MNISTGenerator()

  output shape:(32, 1, 28, 28)
  number of params: 0.0
  output shape:(32, 16, 28, 28)
  number of params: 288
  output shape:(32, 32, 28, 28)
  number of params: 9216
  output shape:(32, 32, 28, 28)
  number of params: 18432
  output shape:(32, 16, 28, 28)
  number of params: 9216
  output shape:(32, 1, 28, 28)
  number of params: 288


/home/mborisyak/opt/anaconda/lib/python2.7/site-packages/lasagne/layers/conv.py:460: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode='full')


In [ ]:
class GAN(object):
    def __init__(self, discriminator_cls, generator_cls):
        X_real = T.ftensor4('X_real')
        
        generator = generator_cls()
        discriminator = discriminator_cls(X_real, generator.X_pseudo)
        
        loss_discriminator = -T.mean(T.log(discriminator.real_predictions)) \
                             -T.mean(T.log(1 - discriminator.pseudo_predictions))
        
        learning_rate = T.fscalar('learning rate')
        
        discriminator_params = layers.get_all_params(discriminator)

        discriminator_updates = updates.adadelta(
            loss_discriminator,
            discriminator_params,
            learning_rate=learning_rate
        )
        
        loss_generator = T.mean(T.log(1 - discriminator.pseudo_predictions))
        
        generator_params = layers.get_all_params(generator)
        generator_updates = updates.adadelta(
            loss_generator,
            generator_params,
            learning_rate=learning_rate
        )
        
        self.train_discriminator = theano.function(
            
        )